In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
day_of_week = spark.sql("SELECT EXTRACT(DAYOFWEEK FROM DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS))").collect()[0][0]

In [0]:
if day_of_week == 5: 
    spark.sql(f"""
        CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_fresh.d_store_group 
        --(
        -- calday DATE,
        -- business_unit STRING,
        -- store_id STRING,
        -- store_name STRING,
        -- region STRING,
        -- base_sale_qty_avg DOUBLE,
        -- sale_kg DOUBLE,
        -- store_group_id STRING,
        -- is_adj STRING,
        -- concept STRING,
        -- qlkv STRING,
        -- email_qlkv STRING,
        -- is_win_plus STRING,
        -- email_gdv STRING, --OPTIONS (DESCRIPTION = 'Email giám đốc vùng'),
        -- city STRING, --OPTIONS (DESCRIPTION = 'Thành phố'),
        -- province STRING, --OPTIONS (DESCRIPTION = 'Tỉnh'),
        -- is_nso STRING, --OPTIONS (DESCRIPTION = 'Có phải NSO hay không, trả về YES/NO'),
        -- origin_store_group_name STRING --OPTIONS (DESCRIPTION = 'Mô tả phân nhóm cửa hàng theo SALE_KG sau điều chỉnh')
        --)
    -- CLUSTER BY store_id
    -- OPTIONS (DESCRIPTION = "Bảng suggest tính phân loại cửa hàng theo sức bán bình quân ngày")
    AS
    WITH main AS (
        SELECT
            ds.business_unit,
            ds.store_id,
            s.store_name,
            s.region,
            s.base_sale_qty_avg,
            s.sale_kg,
            store_group_id,
            CASE
                WHEN ds.business_unit = '1500' THEN 'Supermarket'
                WHEN ds.business_unit = '2000' AND ds.concept IN('WiN','Rural') THEN ds.concept
                ELSE 'Urban'
            END AS concept,
            ds.qlkv,
            ds.email_qlkv,
            ds.is_win_plus,
            ds.email_gdv,
            ds.city,
            ds.region_description AS province,
            'NO' AS is_nso,
            2 AS priority_order
        FROM {catalog_name}.udp_wcm_gold_db_winare_fresh.d_store_group_temp as s
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store ds ON UPPER(s.store_id) = UPPER(ds.store_id)
        WHERE 1=1
            AND ds.store_status_desc IN ('Đang hoạt động', 'Sắp đóng cửa')
            AND ds.business_unit = '2000'
            AND ds.is_win_plus != 'YES'
            AND SUBSTR(ds.store_id, 1, 2) <> 'GH'
        UNION DISTINCT
        SELECT business_unit,
            store_id,
            store_name,
            region,
            base_sale_qty_avg,
            sale_kg,
            store_group_id,
            concept,
            qlkv,
            email_qlkv,
            is_win_plus,
            email_gdv,
            city,
            province,
            is_nso,
            priority_order
        FROM {catalog_name}.udp_wcm_gold_db_winare_fresh.d_store_group_nso_his
        WHERE available_to >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
        QUALIFY ROW_NUMBER() OVER (PARTITION BY store_id ORDER BY proc_date DESC) = 1
    )
    , merge_list AS (
        SELECT DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) calday,
            m.business_unit,
            m.store_id,
            m.store_name,
            m.region,
            m.base_sale_qty_avg,
            m.sale_kg,
            IF(adj.store_group_id IS NOT NULL AND m.store_group_id <> adj.store_group_id, adj.store_group_id, m.store_group_id) store_group_id,
            IF(adj.store_group_id IS NOT NULL AND m.store_group_id <> adj.store_group_id, 'YES', 'NO') is_adj,
            m.concept,
            m.qlkv,
            m.email_qlkv,
            m.is_win_plus,
            m.email_gdv,
            m.city,
            m.province,
            m.is_nso
        FROM main m
        LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_fresh.d_store_storegroup_adj adj ON UPPER(m.store_id) = UPPER(adj.store_id) AND adj.effective_to >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
        WHERE NOT EXISTS(SELECT 1 FROM {catalog_name}.udp_wcm_gold_db_winare_fresh.d_store_exclude_v2 WHERE UPPER(store_id) = UPPER(m.store_id))
        QUALIFY ROW_NUMBER() OVER (PARTITION BY m.store_id ORDER BY m.priority_order) = 1
    )
    SELECT m.* ,
        cf.store_group_name AS origin_store_group_name
    FROM merge_list m
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_fresh.d_cfg_store_group cf ON m.store_group_id = cf.store_group_id AND m.concept = cf.concept;
""")